In [1]:
import os
import sys

DIRECTORY = './'
font = 'arial'

from Library.Duplicate_Model import *

# Duplicate two-sided reactions in a SBML model

A requirement for neural computations with metabolic networks using AMNs is the positivity of all fluxes.

This notebook shows the steps to transforms a SBML model into an *AMN-compatible* SBML model where all exchange reactions are possible in both ways, and reversible internal reactions are duplicated for each way. Added to that, the reactions encoded as backward are recoded in the other way, the forward way. In this transformed model, we also add a suffix "i", for inflowing, and "o" for outflowing reactions. It is further described what we consider "inflow" and "outflow".

# Use the already downloaded model

In [2]:
model_path = "C:/Users/rh2310/AppData/Local/Temp/iML1515.xml"
model = cobra.io.read_sbml_model(model_path)

## Duplicate reactions

In [3]:
io_dict = {"_i": [(None, "e"), (None, "c"), ("e","p"), ("p", "c"), ("e", "c"), ("c", "m"), ("p", "m")],
           "_o": [("c", None), ("e", None), ("p", "e"), ("c", "p"), ("c", "e"), ("m", "c"), ("m", "p")]}

unsignificant_mols = ["h_p", "h_c", "pi_c", "pi_p", "adp_c", "h2o_c", "atp_c"]

# Will print a dictionary counting the reactions in reversible, forward, backward
reac_id_to_io_count_and_way = screen_out_in(model, io_dict, unsignificant_mols)
#
new_model = duplicate_model(model, reac_id_to_io_count_and_way)

{'r': 663, 'f': 2047, 'b': 0, 'o': 2}
The default model had 2712 reactions and the duplicated-reactions model has 3682 reactions.


## Lower bounds check-up

In [4]:
for reac in new_model.reactions:
    if reac.lower_bound != 0:
        print('reaction with non-zero lower bound:', reac.id, reac.bounds)
for el in new_model.medium:
    if new_model.reactions.get_by_id(el).lower_bound != 0:
        print('medium reaction with non-zero lower bound:',el)

reaction with non-zero lower bound: ATPM (6.86, 1000.0)


## Get the default medium back on the duplicated-reactions model

In [5]:
default_med = model.medium
new_med = new_model.medium
correct_med =  correct_duplicated_med(default_med, new_med)
new_model.medium = correct_med

## Medium check-up (default model V.S. duplicated-reaction model)

In [6]:
for i in range(10):
    s, new_s = change_medium(model, new_model, i*3)
    if s != None and new_s != None:
        print(s, new_s, "diff = ", abs(s-new_s))
    elif s != None:
        print("infeasible duplicated medium")
    elif new_s != None:
        print("infeasible default medium")
    elif s == None and new_s == None:
        print("Both medium are impossible")

0.3692496473168983 0.3692496473883942 diff =  7.14959202952059e-11
0.08745529196942409 0.08745529196942273 diff =  1.3600232051658168e-15
0.45251524105941465 0.45251524105941265 diff =  1.9984014443252818e-15
0.3520250542783254 0.35202505427832037 diff =  5.051514762044462e-15
0.7873623549879962 0.7873623549879899 diff =  6.328271240363392e-15
0.25037920551965426 0.2503792055196508 diff =  3.4416913763379853e-15
0.5755530272256282 0.5755530272256252 diff =  2.9976021664879227e-15
0.24708666582568503 0.24708666582568184 diff =  3.191891195797325e-15
0.8737974348363248 0.8737974348363235 diff =  1.2212453270876722e-15
0.7805728235038853 0.7805728235038815 diff =  3.885780586188048e-15


## Saving the duplicated-reactions model

In [7]:
new_model.repair() # rebuild indices and pointers in the model if necessary

In [8]:
from pathlib import Path

print("Original model's location: " + model_path)
model_file = Path(model_path)
new_name = model_file.stem + "_duplicated_Lab_Data" + model_file.suffix

new_name = Path.cwd() / "Dataset_input" / new_name
cobra.io.write_sbml_model(new_model, new_name)
print("Duplicated model's location: " + str(new_name))

Original model's location: C:/Users/rh2310/AppData/Local/Temp/iML1515.xml
Duplicated model's location: C:\Users\rh2310\projects\amn_release\Dataset_input\iML1515_duplicated_Lab_Data.xml
